#OLYMPIC SQUATS ONLY

In [1]:
import os
import numpy as np
import pandas as pd
import sys
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, InputLayer, LSTM, Bidirectional, Reshape
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm
import matplotlib.pyplot as plt
import itertools
import vis_squat as vis
import lstm_model as mod

2023-04-27 19:19:03.118987: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 19:19:03.973070: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/mukundan/anaconda3/envs/aifit/lib/
2023-04-27 19:19:03.973156: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/mukundan/anaconda3/envs/aifit/lib/
2023-04-27 19:19:03.973163: W tensorflow/compi

In [2]:
path_save='/home/mukundan/Desktop/VIII_SEM/Data/labelled_data/media'
visual=vis.VisSquat()
df=visual.get_dat()

100%|██████████| 253/253 [00:00<00:00, 257.51it/s]


In [3]:
key_lst=list(visual.kpts.keys())[:-2]
grp=[]
key_lst_n=[]
for i, key in enumerate(key_lst):
    grp.append(key)
    if i%3==2:
        key_lst_n.append(grp)
        grp=[]
    key_lst=key_lst_n

In [20]:
for num, keys in enumerate(key_lst):
    x_X=[]
    y_X=[]
    z_X=[]
    x_y=[]
    y_y=[]
    z_y=[]
    for key in keys:
        start=0
        end=0
        for i, j in enumerate(list(visual.df['i'])):
            if j==0 and i!=0: 
                end=i-1
                load_key=list(visual.df[key][start+4:end])
                start=end
                if str(key).endswith('x'): 
                    x_X.extend(load_key)
                    x_y.extend(load_key)
                    x_y=x_y[:-1].append(0)
                elif str(key).endswith('y'): 
                    y_X.extend(load_key)
                    y_y.extend(load_key)
                    y_y=x_y[:-1].append(0)
                elif str(key).endswith('z'):
                    z_X.extend(load_key)
                    z_y.extend(load_key)
                    z_y=x_y[:-1].append(0)

    print(x_X, y_X, z_X, x_y, y_y, z_y)

AttributeError: 'NoneType' object has no attribute 'extend'

In [ ]:
for key in df.keys():
    if key=='i' or key=='name': continue
    print('=================================================================')
    print('Training for '+ key)
    print('=================================================================')
    model=mod.get_model.lstm_cell(mod, model_name=key)
    start, end =0,0
    save_name=key+'.h5'
    mc = ModelCheckpoint(save_name, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    for i, j in enumerate(df['i']):
        if j==0 and i!=0: 
            end=i-1
            dat=df[key][start: end]
            start=end
            X=np.array(dat).astype(np.float64).reshape(-1, 1)
            y=dat[:-1]
            y=np.array(y).astype(np.float64).reshape(-1, 1)
            print(X.shape, y.shape)
            y=np.append(y, [-1])

            X_train, X_test, y_train, y_test = train_test_split(X, y,
                                   random_state=104, 
                                   test_size=0.2, shuffle=True)
            
            history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=X.shape[0], epochs=10, verbose=1, shuffle=True, callbacks=[mc])
            loss, acc = model.evaluate(X_test, y_test, verbose=1)
            


In [ ]:
""" #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
#es_va = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)
#mc = ModelCheckpoint('model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=300, epochs=200, verbose=1, shuffle=True)#, callbacks=[mc])
#saved_model = load_model('model.h5')
loss, acc = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', acc) """

In [ ]:
""" y_pred=model.predict(X_test)

'''
for i, y in enumerate(y_test):
    inverted = label_encoder.inverse_transform([np.argmax(y[0:])])
    y_test_inv.append(str(inverted))
    y_test_integer.append(np.argmax(y[0:]))

for i, y in enumerate(y_pred):
    inverted = label_encoder.inverse_transform([np.argmax(y[0:])])
    y_pred_inv.append(str(inverted))
    y_pred_integer.append(np.argmax(y[0:]))
'''
cf_df={'y_pred': y_pred,
       'y_test': y_test}

#cf_df=pd.DataFrame(cf_df)
print(cf_df) """

In [ ]:
plt.plot(history.history['accuracy'], marker='.')
plt.plot(history.history['val_accuracy'], marker='.')
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.grid()
plt.legend(['acc', 'val_acc'], loc='lower right')
#plt.savefig(os.path.join(path_save, 'model_accuracy_lstm_6.png'))
#plt.close()
plt.show()

In [ ]:
plt.plot(history.history['loss'], marker='.')
plt.plot(history.history['val_loss'], marker='.')
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid()
plt.legend(['loss', 'val_loss'], loc='upper right')
#plt.savefig(os.path.join(path_save, 'model_loss_lstm_6.png'))
#plt.close()
plt.show()

In [ ]:
x_test=[-33.098077898188166, -65.44725705933772, 144.44627308952465, -31.77462841751018, -63.828643237381186, 145.78701336608785, -34.1017031670986, -62.85006136716562, 143.65252603902775, -24.533596245715415, -57.87578779807893, 153.0895174999673, -42.44750928025705, -55.90475265338852, 146.70398676454698, -22.847586186716853, -40.706638315604934, 140.63701195494775, -34.386572666179475, -39.654323132533804, 128.7272921663957, -35.930406637552494, -49.72467221807416, 142.39623331564488, -27.250395166471886, -52.83593203381766, 144.07441044344952, -27.36273384675532, -25.061787859943312, 155.6239223417995, -37.876814184041926, -23.666101542744705, 149.1332752310651, -27.686047705438952, -1.5791161838444963, 161.5604528581057, -39.01192009807589, -0.7007367527796415, 152.3842528384619, -26.144767035171707, 19.6444657766348, 160.85365283053812, -41.22631352718785, 20.3417540920933, 158.0220106309576, -27.444019110175738, 22.35788949808039, 158.93113348622, -38.82036501949839, 23.112616602474716, 156.912005070014, -20.5952235212783, 28.88515395203289, 156.93969302396465, -42.17681155370266, 28.502246258244725, 147.1140654570194]
x_test=np.array(x_test).reshape((-1, 19,3))
z=model.predict(x_test)
print(z)